# Import Libary

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install keras

In [ ]:
pip install gdown

In [ ]:
import numpy as np
import gdown
import os
import shutil
import cv2
import seaborn as sns
import zipfile
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load data

In [ ]:
file_id = "1e45f1GmiKKtHkbZPg8WkyjemBVWgjfOE"

# URL berbagi dari Google Drive
url = 'https://drive.google.com/uc?id=' + file_id

# Path untuk menyimpan file yang diunduh
output = 'dataset.zip'

# Mengunduh file
gdown.download(url, output, quiet=False)

# Mengekstrak file zip jika diperlukan
import zipfile

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('datasets/')

Downloading...
From (original): https://drive.google.com/uc?id=1e45f1GmiKKtHkbZPg8WkyjemBVWgjfOE
From (redirected): https://drive.google.com/uc?id=1e45f1GmiKKtHkbZPg8WkyjemBVWgjfOE&confirm=t&uuid=f213586b-41cb-4ae3-8ae2-dd56d1ab5396
To: /content/dataset.zip
100%|██████████| 653M/653M [00:22<00:00, 29.6MB/s]


In [ ]:
def check_and_remove_invalid_images(root_dir):
    invalid_files = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Attempt to open and verify the image
            except Exception as e:
                print(f"Error processing image: {file_path} - {e}")
                invalid_files.append(file_path)
    # Remove invalid files
    for invalid_file in invalid_files:
        os.remove(invalid_file)
        print(f"Invalid image removed: {invalid_file}")

def get_file_paths_and_labels(root_dir):
    # Check and remove invalid images before collecting file paths and labels
    check_and_remove_invalid_images(root_dir)

    file_paths = []
    labels = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            label = os.path.basename(subdir)
            file_paths.append(file_path)
            labels.append(label)
    return file_paths, labels

In [ ]:
def split_and_move_files(file_paths, labels, train_dir, val_dir, test_size=0.2):
    X_train, X_val, y_train, y_val = train_test_split(file_paths, labels, test_size=test_size, stratify=labels, random_state=234)

    for file_path, label in zip(X_train, y_train):
        label_dir = os.path.join(train_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(file_path, label_dir)

    for file_path, label in zip(X_val, y_val):
        label_dir = os.path.join(val_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(file_path, label_dir)

In [ ]:
# Direktori asal, train, dan validation
root_dir = '/content/datasets/'
train_dir = '/content/final-datasets/train'
val_dir = '/content/final-datasets/valid'

# Membuat direktori train dan validation jika belum ada
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Mendapatkan semua jalur file dan labelnya
file_paths, labels = get_file_paths_and_labels(root_dir)

# Membagi dan memindahkan file ke direktori train dan validation
split_and_move_files(
    file_paths,
    labels,
    train_dir,
    val_dir,
    test_size=0.2
    )

Error processing image: /content/datasets/daun kemangi/daun-kemangi_160.jpg - cannot identify image file '/content/datasets/daun kemangi/daun-kemangi_160.jpg'
Error processing image: /content/datasets/daun kemangi/daun-kemangi_180.jpg - cannot identify image file '/content/datasets/daun kemangi/daun-kemangi_180.jpg'
Error processing image: /content/datasets/daun kemangi/daun-kemangi_154.jpg - cannot identify image file '/content/datasets/daun kemangi/daun-kemangi_154.jpg'
Error processing image: /content/datasets/bawang putih/bawang-putih_152.jpg - cannot identify image file '/content/datasets/bawang putih/bawang-putih_152.jpg'
Invalid image removed: /content/datasets/daun kemangi/daun-kemangi_160.jpg
Invalid image removed: /content/datasets/daun kemangi/daun-kemangi_180.jpg
Invalid image removed: /content/datasets/daun kemangi/daun-kemangi_154.jpg
Invalid image removed: /content/datasets/bawang putih/bawang-putih_152.jpg


# Modeling

In [ ]:
# ImageDataGenerator untuk training dan validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Membuat generator untuk training dan validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(31, activation='softmax')
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    batch_size = 32,
)

Found 5049 images belonging to 31 classes.
Found 1263 images belonging to 31 classes.
Epoch 1/10
 44/158 [=======>......................] - ETA: 4:21 - loss: 3.4790 - accuracy: 0.0857

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


158/158 [==============================] - 384s 2s/step - loss: 3.1691 - accuracy: 0.1173 - val_loss: 2.7511 - val_accuracy: 0.1956
Epoch 2/10
158/158 [==============================] - 397s 3s/step - loss: 2.5467 - accuracy: 0.2551 - val_loss: 2.4031 - val_accuracy: 0.2581
Epoch 3/10
158/158 [==============================] - 403s 3s/step - loss: 2.2347 - accuracy: 0.3393 - val_loss: 2.1613 - val_accuracy: 0.3508
Epoch 4/10
158/158 [==============================] - 385s 2s/step - loss: 1.9414 - accuracy: 0.4328 - val_loss: 2.0772 - val_accuracy: 0.3674
Epoch 5/10
158/158 [==============================] - 406s 3s/step - loss: 1.6044 - accuracy: 0.5361 - val_loss: 1.8595 - val_accuracy: 0.4418
Epoch 6/10
158/158 [==============================] - 407s 3s/step - loss: 1.2892 - accuracy: 0.6304 - val_loss: 1.8547 - val_accuracy: 0.4291
Epoch 7/10
158/158 [==============================] - 385s 2s/step - loss: 0.9828 - accuracy: 0.7304 - val_loss: 1.6874 - val_accuracy: 0.5107
Epoch 8/10